In [ ]:
import cr_autophagy as cra

# Imports of general-purpose python libraries
from pathlib import Path

# output_path = cra.get_last_output_path()
output_path = Path("out/autophagy/2023-12-12-T00-34-23")
simulation_settings = cra.get_simulation_settings(output_path)
max_iter = max(cra.get_all_iterations(output_path))

In [ ]:
import importlib
importlib.reload(cra)


discretization_factor = 0.5
bw_method = 0.12
fig = cra.plot_cluster_distribution(output_path, max_iter, discretization_factor, bw_method)

In [ ]:
fig = cra.save_kernel_density(
    output_path,
    max_iter,
    overwrite=True,
    threshold=0.001,
    discretization_factor=0.5,
    bw_method=0.15
)

In [ ]:
# cra.save_all_kernel_density(output_path, threshold=0.6, overwrite=True, bw_method=0.4, discretization_factor=0.5)
# 
# bashcmd = f"ffmpeg -v quiet -stats -y -r 30 -f image2 -pattern_type glob -i '{output_path}/kernel_density/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/kernel_density_movie.mp4"
# os.system(bashcmd)